<a href="https://colab.research.google.com/github/slunara/corporateProject/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Revenue Forecast

In [142]:
import numpy as np
import pandas as pd


def compute_customer_forecast(n_months, traffic, prob_prospect_generation,
                              prob_prospect_conversion,
                              prob_direct_customer_conversion, retention_prob, prob_existing_clients_conversion,
                              existing_customers):
    prospects = np.zeros(n_months)
    new_customers_from_prospects = np.zeros(n_months)
    new_customers_direct = np.zeros(n_months)
    total_existing_customers = np.zeros(n_months)

    prospects = traffic * prob_prospect_generation
    rows, cols = np.meshgrid(np.arange(n_months), np.arange(n_months), indexing="ij")
    M_conversion_matrix_ops = np.where(cols <= rows, prob_prospect_conversion[rows-cols], 0)
    new_customers_from_prospects = M_conversion_matrix_ops @ prospects
    new_customers_direct = traffic * prob_direct_customer_conversion
    M_retention_matrix_ops = np.where(cols <= rows, retention_prob[rows-cols], 0)
    retained_customers_from_prospects = M_retention_matrix_ops @ new_customers_from_prospects
    retained_customers_direct = M_retention_matrix_ops @ new_customers_direct
    existing_customers_vector = np.full(n_months, existing_customers * prob_existing_clients_conversion)
    total_existing_customers = existing_customers_vector + retained_customers_from_prospects + retained_customers_direct
    forecast_df = pd.DataFrame({
    "prospects": prospects,
    "new_customers_from_prospects": new_customers_from_prospects,  # Sum across rows
    "new_customers_direct": new_customers_direct,
    "total_existing_customers": total_existing_customers
    }, index=[f"Month {i+1}" for i in range(n_months)])

    return forecast_df

def compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df):

    total_revenue_df = pd.DataFrame(index=local_customer_forecast_df.index)

    total_revenue_df['revenue_local_new_from_prospects'] = local_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['local_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_local_new_from_direct'] = local_customer_forecast_df['new_customers_direct']*avg_ticket_df['local_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_local_existing'] = local_customer_forecast_df['total_existing_customers']*avg_ticket_df['local_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_local_total'] = total_revenue_df['revenue_local_new_from_prospects']+total_revenue_df['revenue_local_new_from_direct']+total_revenue_df['revenue_local_existing']

    total_revenue_df['revenue_tourist_new_from_prospects'] = tourist_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['tourist_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_tourist_new_from_direct'] = tourist_customer_forecast_df['new_customers_direct']*avg_ticket_df['tourist_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_tourist_existing'] = tourist_customer_forecast_df['total_existing_customers']*avg_ticket_df['tourist_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_tourist_total'] = total_revenue_df['revenue_tourist_new_from_prospects']+total_revenue_df['revenue_tourist_new_from_direct']+total_revenue_df['revenue_tourist_existing']

    total_revenue_df['revenue_total'] = total_revenue_df['revenue_local_total']+total_revenue_df['revenue_tourist_total']

    return total_revenue_df

In [143]:
# ------------------------------ #
# Example usage for Local & Tourist Customers
# ------------------------------ #

n_months = 6
traffic = np.array([1000, 1200, 1100, 1300, 1250, 1400])  # traffic
prob_prospect_generation = 0.30
prob_prospect_conversion = np.array([0.3, 0.2, 0.1, 0, 0, 0])  # 30% of traffic converts into prospects

tourist_prob_prospect_generation = 0

local_prob_direct_customer_conversion = 0.1  # 10% of traffic converts directly into customers
tourist_prob_direct_customer_conversion = 0.05

local_retention_prob = np.array([1.0, 0.7, 0.5, 0.3, 0.2, 0.1])  # Retention probability for new customers
tourist_retention_prob = np.array([0, 0.7, 0.5, 0.3, 0.2, 0.1])  # Retention probability for new customers

local_prob_existing_clients_conversion = 0.03
tourist_prob_existing_clients_conversion = 0.01

existing_local_customers = 5000  # Existing local customers before the forecast
existing_tourist_customers = 1000  # Existing tourist customers before the forecast

local_avg_ticket_new_from_prospects = 50
local_avg_ticket_new_direct = 60
local_avg_ticket_existing = 40

tourist_avg_ticket_new_from_prospects = 0
tourist_avg_ticket_new_direct = 80
tourist_avg_ticket_existing = 45

avg_ticket_df = pd.DataFrame({
    "local_avg_ticket_new_from_prospects": [local_avg_ticket_new_from_prospects],
    "local_avg_ticket_new_direct": [local_avg_ticket_new_direct],
    "local_avg_ticket_existing": [local_avg_ticket_existing],
    "tourist_avg_ticket_new_from_prospects": [tourist_avg_ticket_new_from_prospects],
    "tourist_avg_ticket_new_direct": [tourist_avg_ticket_new_direct],
    "tourist_avg_ticket_existing": [tourist_avg_ticket_existing]
})


# Compute forecasts for Local Customers
local_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, prob_prospect_generation,prob_prospect_conversion,
    local_prob_direct_customer_conversion, local_retention_prob, local_prob_existing_clients_conversion,
    existing_local_customers
)

# Compute forecasts for Tourist Customers
tourist_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, tourist_prob_prospect_generation,  prob_prospect_conversion,
    tourist_prob_direct_customer_conversion, tourist_retention_prob, tourist_prob_existing_clients_conversion,
    existing_tourist_customers
)

total_revenue=compute_revenue_forecast(local_customer_forecast_df,tourist_customer_forecast_df,avg_ticket_df)

In [165]:
total_revenue['revenue_total'].sum()

np.float64(339200.25)

In [145]:
local_customer_forecast_df.sum()

,0
prospects,2175.00
new_customers_from_prospects,1141.50
new_customers_direct,725.00
total_existing_customers,4727.35


In [146]:
tourist_customer_forecast_df.sum()

,0
prospects,0.00
new_customers_from_prospects,0.00
new_customers_direct,362.50
total_existing_customers,456.25


# Sensitivity Analysis

Identify the most influential variable for revenue

In [166]:
variables = {
    "local_prob_prospect_generation": prob_prospect_generation,
    "local_prob_prospect_conversion": prob_prospect_conversion,
    "local_prob_direct_customer_conversion": local_prob_direct_customer_conversion,
    "local_prob_existing_clients_conversion": local_prob_existing_clients_conversion,
    "local_retention_prob":local_retention_prob,

    "tourist_prob_direct_customer_conversion":tourist_prob_direct_customer_conversion,
    "tourist_prob_existing_clients_conversion": tourist_prob_existing_clients_conversion,
    "tourist_retention_prob":tourist_retention_prob
}

In [200]:
# Define baseline total revenue
baseline_total_revenue = total_revenue['revenue_total'].sum()
# Store sensitivity results
sensitivity_results = {}

# Loop through each variable to test its sensitivity
for var in variables.keys():
    # Create a copy of the original variables (fresh start each time)

    modified_variables = {k: v.copy() if isinstance(v, np.ndarray) else v for k, v in variables.items()}


    # Apply a 1% increase ONLY to the current variable
    modified_variables[var] = variables[var] * 1.01  # Increase by 1%


    # Compute forecasts using modified variables
    local_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        modified_variables["local_prob_prospect_generation"],
        modified_variables["local_prob_prospect_conversion"],
        modified_variables["local_prob_direct_customer_conversion"],
        modified_variables["local_retention_prob"],
        modified_variables["local_prob_existing_clients_conversion"],
        existing_local_customers
    )

    tourist_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        tourist_prob_prospect_generation,
        prob_prospect_conversion,
        modified_variables["tourist_prob_direct_customer_conversion"],
        modified_variables["tourist_retention_prob"],
        modified_variables["tourist_prob_existing_clients_conversion"],
        existing_tourist_customers
    )

    # Compute new total revenue
    new_total_revenue_df = compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df)

    # Extract sum of revenue
    new_total_revenue = new_total_revenue_df['revenue_total'].sum()


    # Calculate percentage impact
    sensitivity_results[var] = ((new_total_revenue - baseline_total_revenue) / baseline_total_revenue) * 100

# Convert results to DataFrame and display
sensitivity_df = pd.DataFrame.from_dict(sensitivity_results, orient='index', columns=["% Impact on Total Revenue"])
sensitivity_df = sensitivity_df.sort_values(by="% Impact on Total Revenue", ascending=False)

In [201]:
sensitivity_df

,% Impact on Total Revenue
local_retention_prob,0.451338
local_prob_prospect_generation,0.440651
local_prob_prospect_conversion,0.440651
local_prob_direct_customer_conversion,0.307193
tourist_prob_direct_customer_conversion,0.138064
local_prob_existing_clients_conversion,0.106132
tourist_retention_prob,0.052569
tourist_prob_existing_clients_conversion,0.007960


# Revenue GAP Forecast vs Budget

In [202]:
sales_budget=400000

In [203]:
def compute_new_forecast(sales_budget,sensitivity_df,baseline_total_revenue):

    # Calcular el porcentaje de crecimiento necesario
    growth_required = ((sales_budget - baseline_total_revenue) / baseline_total_revenue) * 100

    print(f"Alert: It is required a growth in sales {growth_required:.2f}% to reach sales budget.")
    incrementPerAction= (0.01*growth_required/sensitivity_df)*100
    incrementPerAction = incrementPerAction.rename(columns={"% Impact on Total Revenue": "Required Percentage Change"})

    return incrementPerAction


In [204]:
incrementPerAction=compute_new_forecast(sales_budget,sensitivity_df,baseline_total_revenue)

Alert: It is required a growth in sales 17.92% to reach sales budget.


In [205]:
incrementPerAction

,Required Percentage Change
local_retention_prob,39.713999
local_prob_prospect_generation,40.677164
local_prob_prospect_conversion,40.677164
local_prob_direct_customer_conversion,58.349088
tourist_prob_direct_customer_conversion,129.827305
local_prob_existing_clients_conversion,168.888194
tourist_retention_prob,340.973011
tourist_prob_existing_clients_conversion,2251.842593


# Compute the new forecast

In [206]:
variable_To_Change = incrementPerAction.index[0]  # Get the first row's index (variable name)
PO_change = incrementPerAction.iloc[0, 0]  # Get the first row's value

print(f"Variable: {variable_To_Change}, Value: {PO_change}")

Variable: local_retention_prob, Value: 39.713999242297035


In [224]:
def compute_new_forecast(variable_To_Change, PO_change):
    """
    Computes the total revenue after modifying ONLY the given variable by PO_change.

    Parameters:
    - variable_To_Change (str): The name of the variable to be modified.
    - PO_change (float): The percentage change to be applied.

    Returns:
    - new_total_revenue (DataFrame): Updated revenue forecast.
    """
    # Create a copy of the original variable
    updated_variable_value = globals()[variable_To_Change] * (1 + PO_change/100)  # Apply % change

    # Debugging print statement
    print(f"🔍 Changing Variable: {variable_To_Change} | Original: {globals()[variable_To_Change]} -> New: {updated_variable_value}")

    # Recalculate forecasts using updated variable
    local_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        prob_prospect_generation if variable_To_Change != "prob_prospect_generation" else updated_variable_value,
        prob_prospect_conversion if variable_To_Change != "prob_prospect_conversion" else updated_variable_value,
        local_prob_direct_customer_conversion if variable_To_Change != "local_prob_direct_customer_conversion" else updated_variable_value,
        local_retention_prob if variable_To_Change != "local_retention_prob" else updated_variable_value,
        local_prob_existing_clients_conversion if variable_To_Change != "local_prob_existing_clients_conversion" else updated_variable_value,
        existing_local_customers if variable_To_Change != "existing_local_customers" else updated_variable_value
    )

    tourist_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        tourist_prob_prospect_generation if variable_To_Change != "tourist_prob_prospect_generation" else updated_variable_value,
        prob_prospect_conversion if variable_To_Change != "prob_prospect_conversion" else updated_variable_value,
        tourist_prob_direct_customer_conversion if variable_To_Change != "tourist_prob_direct_customer_conversion" else updated_variable_value,
        tourist_retention_prob if variable_To_Change != "tourist_retention_prob" else updated_variable_value,
        tourist_prob_existing_clients_conversion if variable_To_Change != "tourist_prob_existing_clients_conversion" else updated_variable_value,
        existing_tourist_customers if variable_To_Change != "existing_tourist_customers" else updated_variable_value
    )

    # Compute new total revenue
    new_total_revenue_df = compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df)

    return new_total_revenue_df,local_customer_forecast_df,tourist_customer_forecast_df


In [225]:
new_total_revenue_df,local_customer_forecast_df,tourist_customer_forecast_df=compute_new_forecast(variable_To_Change, PO_change)
new_total_revenue = new_total_revenue_df['revenue_total'].sum()
new_total_revenue

🔍 Changing Variable: local_retention_prob | Original: [1.  0.7 0.5 0.3 0.2 0.1] -> New: [1.39713999 0.97799799 0.69857    0.419142   0.279428   0.139714  ]


np.float64(400000.0000000022)

In [226]:
new_total_revenue_df.sum()

,0
revenue_local_new_from_prospects,57075.00
revenue_local_new_from_direct,43500.00
revenue_local_existing,249893.75
revenue_local_total,350468.75
revenue_tourist_new_from_prospects,0.00
revenue_tourist_new_from_direct,29000.00
revenue_tourist_existing,20531.25
revenue_tourist_total,49531.25
revenue_total,400000.00


In [227]:
local_customer_forecast_df.sum()

,0
prospects,2175.00000
new_customers_from_prospects,1141.50000
new_customers_direct,725.00000
total_existing_customers,6247.34375


In [228]:
tourist_customer_forecast_df.sum()

,0
prospects,0.00
new_customers_from_prospects,0.00
new_customers_direct,362.50
total_existing_customers,456.25


In [231]:
print(tourist_customer_forecast_df.columns)
print(local_customer_forecast_df.columns)
print(new_total_revenue_df.columns)

Index(['prospects', 'new_customers_from_prospects', 'new_customers_direct',
       'total_existing_customers'],
      dtype='object')
Index(['prospects', 'new_customers_from_prospects', 'new_customers_direct',
       'total_existing_customers'],
      dtype='object')
Index(['revenue_local_new_from_prospects', 'revenue_local_new_from_direct',
       'revenue_local_existing', 'revenue_local_total',
       'revenue_tourist_new_from_prospects', 'revenue_tourist_new_from_direct',
       'revenue_tourist_existing', 'revenue_tourist_total', 'revenue_total'],
      dtype='object')


# KPI re-calculation

In [233]:
import pandas as pd

def compute_kpi(traffic, existing_local_customers, existing_tourist_customers, total_revenue_df, local_customer_forecast_df, tourist_customer_forecast_df):
    """
    Computes key performance indicators (KPIs) based on the given data.

    Parameters:
    - traffic (array): Array containing traffic data per month.
    - existing_local_customers (int): The number of existing local customers before the forecast.
    - existing_tourist_customers (int): The number of existing tourist customers before the forecast.
    - total_revenue_df (DataFrame): DataFrame containing revenue calculations.
    - local_customer_forecast_df (DataFrame): DataFrame containing local customer forecasts.
    - tourist_customer_forecast_df (DataFrame): DataFrame containing tourist customer forecasts.

    Returns:
    - kpi_df (DataFrame): DataFrame containing the computed KPI values.
    """

    # Compute local new closing ratio (new direct customers / total traffic)
    local_new_closing_ratio = local_customer_forecast_df['new_customers_direct'].sum() / traffic.sum()

    # Compute prospect closing ratio (new customers from prospects / total traffic)
    prospect_closing_ratio = local_customer_forecast_df['new_customers_from_prospects'].sum() / traffic.sum()

    # Compute local comeback rate (existing revenue / (existing customers + new direct customers))
    local_come_back = local_customer_forecast_df['total_existing_customers'].sum() / (
        existing_local_customers + local_customer_forecast_df['new_customers_direct'].sum()
    )

    # Compute tourist new closing ratio (new direct tourist customers / total traffic)
    tourist_new_closing_ratio = tourist_customer_forecast_df['new_customers_direct'].sum() / traffic.sum()

    # Compute tourist comeback rate (existing tourist customers / (previous existing tourists + new direct tourists))
    tourist_come_back = tourist_customer_forecast_df['total_existing_customers'].sum() / (
        existing_tourist_customers + tourist_customer_forecast_df['new_customers_direct'].sum()
    )

    # Create a DataFrame to store KPIs
    kpi_df = pd.DataFrame({
        "KPI": ["Local New Closing Ratio", "Prospect Closing Ratio", "Local Comeback Rate", "Tourist New Closing Ratio", "Tourist Comeback Rate"],
        "Value": [local_new_closing_ratio, prospect_closing_ratio, local_come_back, tourist_new_closing_ratio, tourist_come_back]
    })

    return kpi_df

# Example usage
kpi_results = compute_kpi(traffic, existing_local_customers, existing_tourist_customers, new_total_revenue_df, local_customer_forecast_df, tourist_customer_forecast_df)

kpi_results

,KPI,Value
0,Local New Closing Ratio,0.100000
1,Prospect Closing Ratio,0.157448
2,Local Comeback Rate,1.091239
3,Tourist New Closing Ratio,0.050000
4,Tourist Comeback Rate,0.334862


In [163]:
def compute_kpi(local_new_direct,traffic,local_new_from_prospects,local_existing,existing_local_customers,tourist_new_direct,tourist_existing,existing_tourist_customers):
    local_new_closing_ratio=local_new_direct.sum().values[0]/traffic.sum()
    prospect_closing_ratio=local_new_from_prospects.sum().values[0]/traffic.sum()
    local_come_back=local_existing.sum().values[0]/(existing_local_customers+local_existing.sum().values[0])
    tourist_new_closing_ratio=tourist_new_direct.sum().values[0]/traffic.sum()
    tourist_come_back=tourist_existing.sum()/(existing_tourist_customers+tourist_existing.sum().values[0])
    kpi_df=pd.DataFrame([local_new_closing_ratio,prospect_closing_ratio,local_come_back,tourist_new_closing_ratio, tourist_come_back])
    return kpi_df


In [164]:
def compute_kpi(traffic,existing_local_customers,existing_tourist_customers,total_revenue_df):
    local_new_closing_ratio=total_revenue_df['revenue_local_new_from_direct'].sum().values[0]/traffic.sum()
    prospect_closing_ratio=total_revenue_df['revenue_local_new_from_prospects'].values[0]/traffic.sum()
    local_come_back=total_revenue_df['revenue_local_existing'].values[0]/(existing_local_customers+total_revenue_df['revenue_local_new_from_direct'].values[0]+)
    tourist_new_closing_ratio=tourist_new_direct.sum().values[0]/traffic.sum()
    tourist_come_back=tourist_existing.sum()/(existing_tourist_customers+tourist_existing.sum().values[0])

    kpi_df=pd.DataFrame([local_new_closing_ratio,prospect_closing_ratio,local_come_back,tourist_new_closing_ratio, tourist_come_back])
    return kpi_df

kpi_df=compute_kpi(traffic,existing_local_customers,existing_tourist_customers,total_revenue_df)

SyntaxError: invalid syntax (<ipython-input-164-f915514df4a4>, line 4)